In [6]:
quietly {
use OPM_database_clean.dta, clear
}

In [7]:
set more off 

label define yncs 1 "Yes" 0 "No", replace

*Variable Creation*
tab V_PC_None, m
label define VCPENO 1 "VC" 2 "PE" 0 "None", replace
label var V_PC_None VCPENO

*Venture Capital Dummy*
gen VC_dum=.
replace VC_dum=1 if V_PC_None==1
replace VC_dum=0 if V_PC_None==2 | V_PC_None==0
label var VC_dum "Venture capital dum"
 label val VC_dum yncs
tab VC_dum V_PC_None, m

*Private Equity Dummy*
gen PE_dum=.
replace PE_dum=1 if V_PC_None==2
replace PE_dum=0 if V_PC_None==1 | V_PC_None==0
label var PE_dum "Private Equity dum"
 label val PE_dum yncs
tab PE_dum V_PC_None, m

*No Venture Capital/Private Equity Dummy*
gen NoVCPE_dum=.
replace NoVCPE_dum=1 if V_PC_None==0
replace NoVCPE_dum=0 if V_PC_None==1 | V_PC_None==2
label var NoVCPE_dum "No VC/PE dum"
 label val NoVCPE_dum yncs
tab NoVCPE_dum V_PC_None, m

*Venture Capital or Private Equity Dummy*
gen VC_or_PE_dum=.
replace VC_or_PE_dum=1 if V_PC_None==1 | V_PC_None==2
replace VC_or_PE_dum=0 if V_PC_None==0
label var VC_or_PE_dum "PE/VC"
 label val VC_or_PE_dum yncs
tab VC_or_PE_dum V_PC_None, m


order (Institution-Visible) Captivity_AR_INDEF
*Making sure any variables coded as 999 replaced to missing*
foreach var of varlist Start_mo_contract-Captivity_AR_INDEF {
replace `var'=. if `var'==999
}

tab Captivity_AR_INDEF, m

*Creating variable to capture contracts with no rev share/no chg per head setup*
tab Rev_share_w_TP TP_chgs_per_head, m
gen no_rs_ph=.
replace no_rs_ph=1 if Rev_share_w_TP==0 & TP_chgs_per_head==0
replace no_rs_ph=0 if Rev_share_w_TP==1 & TP_chgs_per_head==0
replace no_rs_ph=0 if Rev_share_w_TP==0 & TP_chgs_per_head==1
list Provider Institution if Rev_share_w_TP==0 & TP_chgs_per_head==0

*Visibility*
tab Visible, m
*Visible categorical*
gen Visiblerc=.
replace Visiblerc=0 if Visible==0
replace Visiblerc=1 if Visible==1
replace Visiblerc=2 if Visible==2
replace Visiblerc=3 if Visible==3
replace Visiblerc=4 if Visible==4
tab Visible Visiblerc, m

*Visible category 1*
gen invisible_1=.
replace invisible_1=1 if Visiblerc==0
replace invisible_1=0 if Visiblerc==1 | Visiblerc==2 | Visiblerc==3 | ///
Visiblerc==4
label var invisible_1 "Invisibility"
 label val invisible_1 yncs
tab invisible_1 Visiblerc, m

*Visible category 1 & 2*
gen invisible_2=.
replace invisible_2=1 if Visiblerc==0 | Visiblerc==1 
replace invisible_2=0 if Visiblerc==2 | Visiblerc==3 | ///
Visiblerc==4 
 label val invisible_2 yncs
tab invisible_2 Visiblerc, m

*Visible category 1 & 2 & 3*
gen invisible_3=.
replace invisible_3=0 if Visiblerc==0 | Visiblerc==1 | Visiblerc==2 
replace invisible_3=1 if  Visiblerc==3 | Visiblerc==4 
 label val invisible_3 yncs
tab invisible_3 Visiblerc, m

*Visible category 1 & 2 & 3 & 4* Most visible partnership = reference category*
gen invisible_4=.
replace invisible_4=0 if Visiblerc==0 | Visiblerc==1 | Visiblerc==2 | Visiblerc==3
replace invisible_4=1 if  Visiblerc==4 
 label val invisible_4 yncs
tab invisible_4 Visiblerc, m

*Creating Learning Management identifier*
*Providers identified in paper*
*Angel Learning or Blackboard, Desire2Learn (D2L), Instructure, Longsight, and Moodle.
encode Provider, gen(provider_rc)
tab provider_rc, m
tab provider_rc, m nol

gen LMS=0
replace LMS=1 if provider_rc==8 | provider_rc==15 | ///
provider_rc==29 | provider_rc==48 | provider_rc==49 | provider_rc==58 | ///
provider_rc==62
tab LMS, m


label var Rev_share_w_TP "Revenue Share"
label var TP_recruiting "Recruitment"
label var no_rs_ph "Charges Per Service"
label var TP_chgs_per_head "Charges per Head"
label var Expansion "Expansion"
label var Captivity_AR_INDEF "Captivity"


In [8]:
save "OPM_database_paperJN.dta", replace

file OPM_database_paperJN.dta saved
